### Basic setup of neural network

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import caffe
import os

caffe.set_mode_cpu()
model = 'deploy.prototxt'
weights = 'deep_iter_100000.caffemodel'
net = caffe.Net(model, weights, caffe.TEST)

try:
    os.mkdir("../data/results/")
except OSError: pass
try:
    os.mkdir("../data/results/1/")
except OSError: pass

### Implementation of face detector

    Scaling done by factor of 2 (use of opencv)
    Naive algorithm : every 4 pixel we select a new box to test

In [ ]:
for id_img in range(1,8):
    # Used on 7 images named 1.jpg to 7.jpg
    image_path = '../data/'+str(id_img)+'.jpg'
    
    # Open image on gray scale
    im =  cv2.imread(image_path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    imbase = im
    
    scale = 1
    
    base_save_path = "../data/results/"
    img_base_name = str(id_img) + "_"
    
    # Used to batch save detected faces
    keep = dict()
    
    while len(im) >=36*2 and len(im[0]) >36*2:
        # Downscale of the image using opencv pyramid scaling
        im = cv2.pyrDown(im)
        scale*=2
        
        img_scale_name = img_base_name + str(scale) + "_"
        
        print len(im)
        showarray(im)
        
        # Face detector
        for x in range(0,len(im)-36,4):
            for y in range(0,len(im[0])-36,4):
                img_name = img_scale_name + str(x) + "_" + str(y) + ".png"
                
                # Create a sub image on the original scaled image 
                imtmp = np.array(im [x:x+36, y:y+36])
                
                # Transform the data to be compatible as an entry to the neural network
                im_input = imtmp[np.newaxis, np.newaxis, :, :]/256.0
                
                # Input the data in the NN
                net.blobs['data'].reshape(*im_input.shape)
                net.blobs['data'].data[...] = imtmp
                
                # Compute the data
                output = net.forward()
                if output['prob'][0][1] > 0.99:
                    if output['prob'][0][0] < 0.001:
                        # The sub image is a face we add a bounding box in the original image
                        save_dir = "1/"
                        for i in range (x*scale, (x+36)*scale):
                            imbase[i][y*scale]=255
                            imbase[i][(y+36)*scale]=255
                        for j in range(y*scale, (y+36)*scale):
                            imbase[x*scale][j]=255
                            imbase[(x+36)*scale][j]=255
                        save_path = base_save_path + save_dir + img_name
                        keep[save_path] = imtmp
    
    # We show the original image with the bounding boxes and save it
    showarray(imbase)
    save_results = "../data/results/"+str(id_img)+".jpg"
    cv2.imwrite(save_results, imbase)
    
    # We save the faces we found
    for path,img in keep.iteritems():
        cv2.imwrite(path, img)